In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [2]:
!cp /content/drive/MyDrive/Dataset/FaceClasses.zip ./face_data.zip
!unzip -q face_data.zip -d face_data
!find face_data -maxdepth 2 -type d

face_data
face_data/FaceClasses
face_data/FaceClasses/1
face_data/FaceClasses/3
face_data/FaceClasses/0
face_data/FaceClasses/2


In [3]:
import os
class_root = "face_data/FaceClasses"


for class_name in sorted(os.listdir(class_root)):
    class_path = os.path.join(class_root, class_name)
    if os.path.isdir(class_path):
        count = len([f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))])
        print(f"Class {class_name}: {count} images")


Class 0: 3092 images
Class 1: 2909 images
Class 2: 2351 images
Class 3: 3109 images


In [6]:
import os, random
from pathlib import Path
from collections import Counter

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import datasets, models, transforms
from torchvision.models import ResNet50_Weights
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# ─── CONFIG ────────────────────────────────────────────────────────────────
DATA_ROOT         = "face_data/FaceClasses"
BATCH_SIZE        = 32
NUM_EPOCHS        = 25
LR_HEAD           = 1e-4
LR_BACKBONE       = 1e-5
WEIGHT_DECAY      = 1e-4
DEVICE            = torch.device("cuda" if torch.cuda.is_available() else "cpu")
RANDOM_SEED       = 42
NUM_CLASSES       = len(os.listdir(DATA_ROOT))
BEST_MODEL_PATH   = "/content/drive/MyDrive/Secure-Inference/models/resnet50_mod.pth"

torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# ─── TRANSFORMS ────────────────────────────────────────────────────────────
train_tf = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.6,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2,0.2,0.2,0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
val_tf = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

# ─── LOAD & SPLIT ──────────────────────────────────────────────────────────
full = datasets.ImageFolder(DATA_ROOT)
paths, labels = zip(*full.samples)
train_paths, val_paths, train_labels, val_labels = train_test_split(
    paths, labels, test_size=0.2, stratify=labels, random_state=RANDOM_SEED
)

class SubsetFolder(datasets.ImageFolder):
    def __init__(self, root, file_list, transform):
        super().__init__(root, transform=transform)
        self.samples = [(p, self.class_to_idx[Path(p).parent.name]) for p in file_list]
        self.targets = [lbl for _, lbl in self.samples]

train_ds = SubsetFolder(DATA_ROOT, train_paths, train_tf)
val_ds   = SubsetFolder(DATA_ROOT, val_paths,   val_tf)

# ─── BALANCED SAMPLER ──────────────────────────────────────────────────────
counts = Counter(train_ds.targets)
print("Class counts:", counts)
class_weights = {cls: 1.0/count for cls, count in counts.items()}
sample_weights = [class_weights[t] for t in train_ds.targets]
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)
print("Per-class sampling weights:", class_weights)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler,
                          num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=4, pin_memory=True)

# ─── MODEL SETUP ──────────────────────────────────────────────────────────
model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

# 1) Freeze everything
for p in model.parameters():
    p.requires_grad = False

# 2) Unfreeze last two bottleneck blocks (layer3 & layer4) and layer2
for name, p in model.named_parameters():
    if name.startswith(('layer2', 'layer3', 'layer4')):
        p.requires_grad = True

# 3) Replace the head
model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
model = model.to(DEVICE)

# ─── LOSS, OPTIMIZER, SCHEDULER ───────────────────────────────────────────
criterion = nn.CrossEntropyLoss()
opt_params = [
    {'params': model.fc.parameters(),      'lr': LR_HEAD},
    {'params': [p for n,p in model.named_parameters() if p.requires_grad and not n.startswith('fc')],
     'lr': LR_BACKBONE},
]
optimizer = optim.Adam(opt_params, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

# ─── TRAIN / EVAL LOOP ────────────────────────────────────────────────────
best_val_acc = 0.0

for epoch in range(1, NUM_EPOCHS+1):
    # — Train —
    model.train()
    running_loss = running_corrects = 0
    for x, y in tqdm(train_loader, desc=f"Train {epoch}/{NUM_EPOCHS}"):
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x.size(0)
        running_corrects += (out.argmax(1) == y).sum().item()

    scheduler.step()
    train_loss = running_loss / len(train_ds)
    train_acc  = running_corrects / len(train_ds) * 100

    # — Validate —
    model.eval()
    val_corrects = 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            val_corrects += (model(x).argmax(1) == y).sum().item()
    val_acc = val_corrects / len(val_ds) * 100

    print(f"\nEpoch {epoch:02d}  TrainLoss {train_loss:.4f}  TrainAcc {train_acc:.2f}%  ValAcc {val_acc:.2f}%")

    # — Save best —
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_train_acc_for_resnet = train_acc
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        print(f" -> New model (ValAcc={val_acc:.2f}%), saved to {BEST_MODEL_PATH}\n")

print(f"Training complete. Best Val Acc: {best_val_acc:.2f}%. Corresponding Train Acc: {best_train_acc_for_resnet:.2f}%")


Class counts: Counter({3: 2487, 0: 2473, 1: 2327, 2: 1881})
Per-class sampling weights: {3: 0.0004020908725371934, 1: 0.0004297378599054577, 2: 0.000531632110579479, 0: 0.0004043671653861706}


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Train 1/25: 100%|██████████| 287/287 [01:33<00:00,  3.07it/s]



Epoch 01  TrainLoss 1.2516  TrainAcc 46.58%  ValAcc 55.91%
 -> New model (ValAcc=55.91%), saved to /content/drive/MyDrive/Secure-Inference/models/resnet50_mod.pth



Train 2/25: 100%|██████████| 287/287 [01:26<00:00,  3.31it/s]



Epoch 02  TrainLoss 0.9714  TrainAcc 59.98%  ValAcc 67.38%
 -> New model (ValAcc=67.38%), saved to /content/drive/MyDrive/Secure-Inference/models/resnet50_mod.pth



...


Epoch 23  TrainLoss 0.3078  TrainAcc 87.85%  ValAcc 86.79%
 -> New model (ValAcc=86.79%), saved to /content/drive/MyDrive/Secure-Inference/models/resnet50_mod.pth



Train 24/25: 100%|██████████| 287/287 [01:25<00:00,  3.35it/s]



Epoch 24  TrainLoss 0.3006  TrainAcc 89.09%  ValAcc 86.61%


Train 25/25: 100%|██████████| 287/287 [01:23<00:00,  3.42it/s]



Epoch 25  TrainLoss 0.2984  TrainAcc 88.60%  ValAcc 86.22%
Training complete. Best Val Acc: 86.79%. Corresponding Train Acc: 87.85%


In [7]:
import os, random
from pathlib import Path
from collections import Counter

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import datasets, models, transforms
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# ─── CONFIG ────────────────────────────────────────────────────────────────
DATA_ROOT       = "face_data/FaceClasses"
BATCH_SIZE      = 32
NUM_EPOCHS      = 25
LR_HEAD         = 1e-3
LR_BACKBONE     = 1e-4
WEIGHT_DECAY    = 1e-4
DEVICE          = torch.device("cuda" if torch.cuda.is_available() else "cpu")
RANDOM_SEED     = 42
NUM_CLASSES     = len(os.listdir(DATA_ROOT))
BEST_MODEL_PATH = "/content/drive/MyDrive/Secure-Inference/models/squeezenet_mod.pth"

torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# ─── TRANSFORMS ────────────────────────────────────────────────────────────
train_tf = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.6,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2,0.2,0.2,0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
val_tf = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

# ─── LOAD & SPLIT ──────────────────────────────────────────────────────────
full = datasets.ImageFolder(DATA_ROOT)
paths, labels = zip(*full.samples)
train_paths, val_paths, train_labels, val_labels = train_test_split(
    paths, labels, test_size=0.2, stratify=labels, random_state=RANDOM_SEED
)

class SubsetFolder(datasets.ImageFolder):
    def __init__(self, root, file_list, transform):
        super().__init__(root, transform=transform)
        self.samples = [(p, self.class_to_idx[Path(p).parent.name]) for p in file_list]
        self.targets = [lbl for _, lbl in self.samples]

train_ds = SubsetFolder(DATA_ROOT, train_paths, train_tf)
val_ds   = SubsetFolder(DATA_ROOT, val_paths,   val_tf)

# ─── SAMPLER ───────────────────────────────────────────────────────────────
counts = Counter(train_ds.targets)
print("Class counts:", counts)
class_weights = {cls: 1.0/count for cls, count in counts.items()}
sample_weights = [class_weights[t] for t in train_ds.targets]
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)
print("Per-class sampling weights:", class_weights)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler,
                          num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=4, pin_memory=True)

# ─── MODEL SETUP ──────────────────────────────────────────────────────────
model = models.squeezenet1_1(pretrained=True)

# Freeze all
for p in model.parameters():
    p.requires_grad = False

# Unfreeze Fire modules 7 & 8 for more capacity
for name, p in model.named_parameters():
    if name.startswith(("features.12", "features.14")):  # fire7 & fire8
        p.requires_grad = True

# Replace classifier
model.classifier = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Conv2d(512, NUM_CLASSES, kernel_size=1),
    nn.ReLU(inplace=True),
    nn.AdaptiveAvgPool2d((1,1))
)
model = model.to(DEVICE)

# ─── LOSS, OPT, SCHED ──────────────────────────────────────────────────────
criterion = nn.CrossEntropyLoss()
opt_params = [
    {'params': model.classifier.parameters(), 'lr': LR_HEAD},
    {'params': [p for n,p in model.named_parameters() if p.requires_grad and "classifier" not in n],
     'lr': LR_BACKBONE},
]
optimizer = optim.Adam(opt_params, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

# ─── TRAIN & VALIDATE ──────────────────────────────────────────────────────
best_val_acc = 0.0

for epoch in range(1, NUM_EPOCHS+1):
    model.train()
    run_loss = run_corr = 0
    for x,y in tqdm(train_loader, desc=f"Train {epoch}/{NUM_EPOCHS}"):
        x,y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        out = model(x)
        out = out.view(out.size(0), NUM_CLASSES)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        run_loss += loss.item() * x.size(0)
        run_corr += (out.argmax(1)==y).sum().item()

    scheduler.step()
    train_loss = run_loss/len(train_ds)
    train_acc  = run_corr/len(train_ds)*100

    model.eval()
    val_corr = 0
    with torch.no_grad():
        for x,y in val_loader:
            x,y = x.to(DEVICE), y.to(DEVICE)
            out = model(x).view(x.size(0), NUM_CLASSES)
            val_corr += (out.argmax(1)==y).sum().item()
    val_acc = val_corr/len(val_ds)*100

    print(f"\nEpoch {epoch:02d}  TrainLoss {train_loss:.4f}  TrainAcc {train_acc:.2f}%  ValAcc {val_acc:.2f}%")
    if val_acc>best_val_acc:
        best_val_acc = val_acc
        best_train_acc_for_sqnet = train_acc
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        print(f" → New model (ValAcc={val_acc:.2f}%), saved to {BEST_MODEL_PATH}\n")

print(f"Training complete. Best Val Acc: {best_val_acc:.2f}%. Corresponding Train Acc: {best_train_acc_for_sqnet:.2f}%")

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=

Class counts: Counter({3: 2487, 0: 2473, 1: 2327, 2: 1881})
Per-class sampling weights: {3: 0.0004020908725371934, 1: 0.0004297378599054577, 2: 0.000531632110579479, 0: 0.0004043671653861706}


100%|██████████| 4.73M/4.73M [00:00<00:00, 64.1MB/s]
Train 1/25: 100%|██████████| 287/287 [01:09<00:00,  4.11it/s]



Epoch 01  TrainLoss 1.0779  TrainAcc 55.26%  ValAcc 62.36%
 → New model (ValAcc=62.36%), saved to /content/drive/MyDrive/Secure-Inference/models/squeezenet_mod.pth



Train 2/25: 100%|██████████| 287/287 [01:09<00:00,  4.12it/s]


...


Epoch 23  TrainLoss 0.4919  TrainAcc 81.52%  ValAcc 80.68%


Train 24/25: 100%|██████████| 287/287 [01:09<00:00,  4.15it/s]



Epoch 24  TrainLoss 0.4912  TrainAcc 81.54%  ValAcc 80.85%


Train 25/25: 100%|██████████| 287/287 [01:09<00:00,  4.12it/s]



Epoch 25  TrainLoss 0.4968  TrainAcc 81.16%  ValAcc 80.94%
 → New model (ValAcc=80.94%), saved to /content/drive/MyDrive/Secure-Inference/models/squeezenet_mod.pth

Training complete. Best Val Acc: 80.94%. Corresponding Train Acc: 81.16%


In [8]:
!pip install onnx onnxruntime-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.2/283.2 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.1 MB/s eta 0:00:00


In [10]:
import torch, os

def export_onnx(model, dummy_input, path, opset=17, half=False):
    if half:
        model = model.half()
        dummy_input = dummy_input.half()
    torch.onnx.export(
        model, dummy_input, path,
        input_names=["input"], output_names=["output"],
        opset_version=opset,
        dynamic_axes = None,
    )
    size_mb = os.path.getsize(path) / (1024*1024)
    print(f"Exported {path} ({size_mb:.1f} MB)")

# ───── Load Models ──────────────────────────────────────────────────
device = "cpu"

# ResNet50
resnet = models.resnet50(weights=None, num_classes=NUM_CLASSES)
resnet.load_state_dict(torch.load("/content/drive/MyDrive/Secure-Inference/models/resnet50_mod.pth", map_location=device))
resnet.eval()

# SqueezeNet
sq = models.squeezenet1_1(weights=None)
# same classifier override as training
sq.classifier = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Conv2d(512, NUM_CLASSES, 1),
    nn.ReLU(inplace=True),
    nn.AdaptiveAvgPool2d((1,1))
)
sq.load_state_dict(torch.load("/content/drive/MyDrive/Secure-Inference/models/squeezenet_mod.pth", map_location=device))
sq.eval()

# ───── Dummy Input ───────────────────────────────────────────────────────────
dummy = torch.randn(1,3,224,224, device=device)

# 1. ResNet50 FP32
export_onnx(resnet, dummy,    "/content/drive/MyDrive/Secure-Inference/models/resnet50_fp32.onnx", half=False)

# 2. ResNet50 FP16
export_onnx(resnet, dummy,    "/content/drive/MyDrive/Secure-Inference/models/resnet50_fp16.onnx", half=True)

# 3. SqueezeNet FP32
export_onnx(sq,     dummy,    "/content/drive/MyDrive/Secure-Inference/models/sqnet_fp32.onnx",   half=False)

# 4. SqueezeNet FP16
export_onnx(sq,     dummy,    "/content/drive/MyDrive/Secure-Inference/models/sqnet_fp16.onnx",   half=True)


Exported /content/drive/MyDrive/Secure-Inference/models/resnet50_fp32.onnx (89.6 MB)
Exported /content/drive/MyDrive/Secure-Inference/models/resnet50_fp16.onnx (44.8 MB)
Exported /content/drive/MyDrive/Secure-Inference/models/sqnet_fp32.onnx (2.8 MB)
Exported /content/drive/MyDrive/Secure-Inference/models/sqnet_fp16.onnx (1.4 MB)


In [11]:
import os
import numpy as np
import onnxruntime as rt
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from pathlib import Path



ONNX_EVAL_BATCH_SIZE = 1

onnx_eval_transform = val_tf

try:
    val_ds_for_onnx = SubsetFolder(root=DATA_ROOT,
                                   file_list=val_paths,
                                   transform=onnx_eval_transform)

    onnx_test_loader = DataLoader(
        val_ds_for_onnx,
        batch_size=ONNX_EVAL_BATCH_SIZE,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )
    print(f"ONNX Test Loader created with {len(val_ds_for_onnx)} samples (using existing val_paths and SubsetFolder).")
except NameError as e:
    onnx_test_loader = None


def eval_onnx_gpu(model_path, loader):
    if loader is None:
        print(f"Skipping evaluation for {os.path.basename(model_path)} due to loader setup issues.")
        return 0.0

    providers = rt.get_available_providers()
    preferred_providers = []
    if 'CUDAExecutionProvider' in providers: preferred_providers.append('CUDAExecutionProvider')
    if 'CPUExecutionProvider' in providers: preferred_providers.append('CPUExecutionProvider')
    if not preferred_providers: raise RuntimeError("No CUDA or CPU ONNX Execution Provider found.")

    print(f"Loading ONNX model {os.path.basename(model_path)} with providers: {preferred_providers}")
    sess = rt.InferenceSession(model_path, providers=preferred_providers)
    print(f"Using ONNX provider: {sess.get_providers()}")

    inp_meta = sess.get_inputs()[0]
    inp_name = inp_meta.name
    elem_type_str = rt.OrtTensorType.elem_type_to_string(inp_meta.type.tensor_type.elem_type) if hasattr(inp_meta.type, 'tensor_type') else inp_meta.type
    expects_fp16 = "float16" in elem_type_str.lower()

    correct = total = 0
    for images, labels in tqdm(loader, desc=os.path.basename(model_path)):
        arr = images.numpy()
        if expects_fp16 and arr.dtype != np.float16:
            arr = arr.astype(np.float16)
        elif not expects_fp16 and arr.dtype != np.float32:
            arr = arr.astype(np.float32)

        ort_inputs = {inp_name: arr}
        logits = sess.run(None, ort_inputs)[0]
        preds  = np.argmax(logits, axis=1)
        correct += (preds == labels.numpy()).sum()
        total   += labels.size(0)

    return (correct / total * 100) if total > 0 else 0.0

# ONNX Model Paths
onnx_model_configs = {
    "ResNet50 FP32":   "/content/drive/MyDrive/Secure-Inference/models/resnet50_fp32.onnx",
    "ResNet50 FP16":   "/content/drive/MyDrive/Secure-Inference/models/resnet50_fp16.onnx",
    "SqueezeNet FP32": "/content/drive/MyDrive/Secure-Inference/models/sqnet_fp32.onnx",
    "SqueezeNet FP16": "/content/drive/MyDrive/Secure-Inference/models/sqnet_fp16.onnx",
}

onnx_results = {}
if onnx_test_loader:
    for name, path in onnx_model_configs.items():
        print(f"\nEvaluating {name}...")
        assert os.path.isfile(path), f"ONNX Model file missing: {path}"
        onnx_results[name] = eval_onnx_gpu(path, onnx_test_loader)

print("\n--- ONNX Model Accuracies on Test Set ---")
for name, acc in onnx_results.items():
    print(f"  {name:20s} → {acc:.2f}%")

ONNX Test Loader created with 2293 samples (using existing val_paths and SubsetFolder).

Evaluating ResNet50 FP32...
Loading ONNX model resnet50_fp32.onnx with providers: ['CUDAExecutionProvider', 'CPUExecutionProvider']
Using ONNX provider: ['CUDAExecutionProvider', 'CPUExecutionProvider']


resnet50_fp32.onnx: 100%|██████████| 2293/2293 [00:16<00:00, 138.63it/s]



Evaluating ResNet50 FP16...
Loading ONNX model resnet50_fp16.onnx with providers: ['CUDAExecutionProvider', 'CPUExecutionProvider']
Using ONNX provider: ['CUDAExecutionProvider', 'CPUExecutionProvider']


resnet50_fp16.onnx: 100%|██████████| 2293/2293 [00:15<00:00, 148.48it/s]



Evaluating SqueezeNet FP32...
Loading ONNX model sqnet_fp32.onnx with providers: ['CUDAExecutionProvider', 'CPUExecutionProvider']
Using ONNX provider: ['CUDAExecutionProvider', 'CPUExecutionProvider']


sqnet_fp32.onnx: 100%|██████████| 2293/2293 [00:11<00:00, 196.43it/s]



Evaluating SqueezeNet FP16...
Loading ONNX model sqnet_fp16.onnx with providers: ['CUDAExecutionProvider', 'CPUExecutionProvider']
Using ONNX provider: ['CUDAExecutionProvider', 'CPUExecutionProvider']


sqnet_fp16.onnx: 100%|██████████| 2293/2293 [00:13<00:00, 174.54it/s]


--- ONNX Model Accuracies on Test Set ---
  ResNet50 FP32        → 86.79%
  ResNet50 FP16        → 86.70%
  SqueezeNet FP32      → 80.94%
  SqueezeNet FP16      → 80.90%


In [13]:
import os
import random
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from PIL import Image
import numpy as np
import onnxruntime as rt
from pathlib import Path

# ─── CONFIGURATIONS ─────────────────────────────────────────────────────────
DRIVE_MODEL_DIR = "/content/drive/MyDrive/Secure-Inference/models/"
DATA_ROOT = "face_data/FaceClasses"
NUM_CLASSES = 4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMAGES_PER_CLASS_TO_SAMPLE = 2

RESULTS_DIR = "/content/drive/MyDrive/Secure-Inference/phase1_results"
os.makedirs(RESULTS_DIR, exist_ok=True)
INFERENCE_LOG_FILE = os.path.join(RESULTS_DIR, "phase1_non_secure_sample_inference.txt")


if os.path.exists(INFERENCE_LOG_FILE):
    os.remove(INFERENCE_LOG_FILE)

def log_inference(message):
    print(message)
    with open(INFERENCE_LOG_FILE, "a") as f:
        f.write(message + "\n")

log_inference(f"Phase 1 Sample Inference Logging Started. Results will be saved to: {INFERENCE_LOG_FILE}")
log_inference(f"Using device: {DEVICE}")
log_inference(f"DATA_ROOT for sample images: {DATA_ROOT}")
log_inference(f"Number of classes (NUM_CLASSES): {NUM_CLASSES}")

# ─── TRANSFORMS
if 'val_tf' not in locals():
    log_inference("Warning: 'val_tf' not in scope. Redefining standard validation transform.")
    val_tf = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
else:
    log_inference("'val_tf' (validation transform) found in scope and will be used.")


# ─── LOAD PYTORCH MODELS ───────────────────────────────────────────────────
resnet_pytorch = models.resnet50(weights=None, num_classes=NUM_CLASSES)
resnet_pytorch.load_state_dict(torch.load(os.path.join(DRIVE_MODEL_DIR, "resnet50_mod.pth"), map_location=DEVICE))
resnet_pytorch.eval().to(DEVICE)
log_inference(f"ResNet50 PyTorch model loaded from {os.path.join(DRIVE_MODEL_DIR, 'resnet50_mod.pth')}")

# SqueezeNet
sqnet_pytorch = models.squeezenet1_1(num_classes=NUM_CLASSES)

sqnet_pytorch.classifier = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Conv2d(512, NUM_CLASSES, kernel_size=1),
    nn.ReLU(inplace=True),
    nn.AdaptiveAvgPool2d((1, 1))
)
sqnet_pytorch.load_state_dict(torch.load(os.path.join(DRIVE_MODEL_DIR, "squeezenet_mod.pth"), map_location=DEVICE))
sqnet_pytorch.eval().to(DEVICE)
log_inference(f"SqueezeNet PyTorch model loaded from {os.path.join(DRIVE_MODEL_DIR, 'squeezenet_mod.pth')}")


pytorch_model_references = {
    "ResNet50": resnet_pytorch,
    "SqueezeNet": sqnet_pytorch
}

# ─── LOAD ONNX MODELS (SESSIONS) ───────────────────────────────────────────
onnx_model_file_paths = {
    "ResNet50_FP32": os.path.join(DRIVE_MODEL_DIR, "resnet50_fp32.onnx"),
    "ResNet50_FP16": os.path.join(DRIVE_MODEL_DIR, "resnet50_fp16.onnx"),
    "SqueezeNet_FP32": os.path.join(DRIVE_MODEL_DIR, "sqnet_fp32.onnx"),
    "SqueezeNet_FP16": os.path.join(DRIVE_MODEL_DIR, "sqnet_fp16.onnx"),
}

onnx_sessions = {}
for name, path in onnx_model_file_paths.items():
    if not os.path.exists(path):
        log_inference(f"ERROR: ONNX model path not found: {path}")
        continue
    try:
        providers = rt.get_available_providers()
        preferred_providers = []
        if 'CUDAExecutionProvider' in providers: preferred_providers.append('CUDAExecutionProvider')
        if 'CPUExecutionProvider' in providers: preferred_providers.append('CPUExecutionProvider')
        if not preferred_providers: raise RuntimeError("No suitable ONNX Execution Provider found.")

        onnx_sessions[name] = rt.InferenceSession(path, providers=preferred_providers)
        log_inference(f"ONNX session created for {name} using {onnx_sessions[name].get_providers()}")
    except Exception as e:
        log_inference(f"ERROR: Failed to create ONNX session for {name}. Error: {e}")

def infer_pytorch(model, image_tensor_batch):
    with torch.no_grad():
        logits = model(image_tensor_batch.to(DEVICE))

        if isinstance(model, models.SqueezeNet) and logits.dim() > 2 :
             logits = logits.view(logits.size(0), NUM_CLASSES)

        probabilities = torch.softmax(logits, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1)
    return logits.cpu().numpy(), probabilities.cpu().numpy(), predicted_class.cpu().numpy()

def infer_onnx(session, image_numpy_batch):
    inp_meta = session.get_inputs()[0]
    inp_name = inp_meta.name

    elem_type_str = rt.OrtTensorType.elem_type_to_string(inp_meta.type.tensor_type.elem_type) if hasattr(inp_meta.type, 'tensor_type') else inp_meta.type
    expects_fp16 = "float16" in elem_type_str.lower()

    if expects_fp16 and image_numpy_batch.dtype != np.float16:
        image_numpy_batch = image_numpy_batch.astype(np.float16)
    elif not expects_fp16 and image_numpy_batch.dtype != np.float32:
         image_numpy_batch = image_numpy_batch.astype(np.float32)

    ort_inputs = {inp_name: image_numpy_batch}
    logits = session.run(None, ort_inputs)[0]

    exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
    probabilities = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)
    predicted_class = np.argmax(probabilities, axis=1)
    return logits, probabilities, predicted_class

FIXED_SAMPLE_IMAGES_FILENAMES = {
    "0": ["aa048t2aaunaff005.png", "aa048t2aaunaff008.png"],
    "1": ["aa048t2aeaff017.png", "aa048t2aeaff018.png"],
    "2": ["ak064t1aaaff100.png", "ak064t1aaaff101.png"],
    "3": ["ak064t1aaaff095.png", "ak064t1aaaff096.png"]
}


log_inference(f"\nUsing FIXED sample images: {FIXED_SAMPLE_IMAGES_FILENAMES}")
collected_sample_image_paths = {}
for class_name_str, filenames_in_class in FIXED_SAMPLE_IMAGES_FILENAMES.items():
    current_class_paths = []
    class_dir = os.path.join(DATA_ROOT, class_name_str)
    if not os.path.isdir(class_dir):
        log_inference(f"Warning: Sample image class directory not found: {class_dir}. Skipping class {class_name_str}.")
        continue
    for img_filename in filenames_in_class:
        img_full_path = os.path.join(class_dir, img_filename)
        if os.path.isfile(img_full_path):
            current_class_paths.append(img_full_path)
        else:
            log_inference(f"Warning: Specified sample image NOT FOUND: {img_full_path}. Skipping this image.")
    if current_class_paths:
        collected_sample_image_paths[class_name_str] = current_class_paths

if not collected_sample_image_paths:
    log_inference("ERROR: No valid fixed sample images were found based on FIXED_SAMPLE_IMAGES_FILENAMES. Check paths and DATA_ROOT.")
else:
    log_inference(f"Verified sample images to be processed: {collected_sample_image_paths}")

# ─── PERFORM AND LOG INFERENCE ─────────────────────────────────────────────
log_inference("\n" + "="*30 + " SAMPLE INFERENCE RESULTS " + "="*30)

if not collected_sample_image_paths:
    log_inference("Skipping inference as no sample images were loaded.")
else:
    for class_name_str, image_paths_list in collected_sample_image_paths.items():
        true_label_int = int(class_name_str)

        for img_path in image_paths_list:
            log_inference(f"\n--- Image: {img_path} (True Label: {true_label_int}) ---")
            try:
                image_pil = Image.open(img_path).convert("RGB")
                image_tensor = val_tf(image_pil)
                image_tensor_batch = image_tensor.unsqueeze(0)
                image_numpy_batch = image_tensor_batch.cpu().numpy()

                # --- PyTorch Inference ---
                # ResNet50 PyTorch
                pt_model_resnet = pytorch_model_references["ResNet50"]
                pt_logits, pt_probs, pt_pred = infer_pytorch(pt_model_resnet, image_tensor_batch)
                log_inference(f"  [PyTorch ResNet50]:")
                log_inference(f"    Predicted Class: {pt_pred[0]}, Probabilities: {np.array2string(pt_probs[0], precision=4)}")
                log_inference(f"    Logits: {np.array2string(pt_logits[0], precision=4)}")

                # SqueezeNet PyTorch
                pt_model_sqnet = pytorch_model_references["SqueezeNet"]
                pt_logits, pt_probs, pt_pred = infer_pytorch(pt_model_sqnet, image_tensor_batch)
                log_inference(f"  [PyTorch SqueezeNet]:")
                log_inference(f"    Predicted Class: {pt_pred[0]}, Probabilities: {np.array2string(pt_probs[0], precision=4)}")
                log_inference(f"    Logits: {np.array2string(pt_logits[0], precision=4)}")

                # --- ONNX Inference ---
                # ResNet50 ONNX models
                if "ResNet50_FP32" in onnx_sessions:
                    onnx_logits, onnx_probs, onnx_pred = infer_onnx(onnx_sessions["ResNet50_FP32"], image_numpy_batch.copy())
                    log_inference(f"  [ONNX ResNet50_FP32]:")
                    log_inference(f"    Predicted Class: {onnx_pred[0]}, Probabilities: {np.array2string(onnx_probs[0], precision=4)}")
                    log_inference(f"    Logits: {np.array2string(onnx_logits[0], precision=4)}")
                if "ResNet50_FP16" in onnx_sessions:
                    onnx_logits, onnx_probs, onnx_pred = infer_onnx(onnx_sessions["ResNet50_FP16"], image_numpy_batch.copy())
                    log_inference(f"  [ONNX ResNet50_FP16]:")
                    log_inference(f"    Predicted Class: {onnx_pred[0]}, Probabilities: {np.array2string(onnx_probs[0], precision=4)}")
                    log_inference(f"    Logits: {np.array2string(onnx_logits[0], precision=4)}")

                # SqueezeNet ONNX models
                if "SqueezeNet_FP32" in onnx_sessions:
                    onnx_logits, onnx_probs, onnx_pred = infer_onnx(onnx_sessions["SqueezeNet_FP32"], image_numpy_batch.copy())
                    log_inference(f"  [ONNX SqueezeNet_FP32]:")
                    log_inference(f"    Predicted Class: {onnx_pred[0]}, Probabilities: {np.array2string(onnx_probs[0], precision=4)}")
                    log_inference(f"    Logits: {np.array2string(onnx_logits[0], precision=4)}")
                if "SqueezeNet_FP16" in onnx_sessions:
                    onnx_logits, onnx_probs, onnx_pred = infer_onnx(onnx_sessions["SqueezeNet_FP16"], image_numpy_batch.copy())
                    log_inference(f"  [ONNX SqueezeNet_FP16]:")
                    log_inference(f"    Predicted Class: {onnx_pred[0]}, Probabilities: {np.array2string(onnx_probs[0], precision=4)}")
                    log_inference(f"    Logits: {np.array2string(onnx_logits[0], precision=4)}")

            except Exception as e:
                log_inference(f"  ERROR processing image {img_path}: {e}")
                import traceback
                log_inference(traceback.format_exc())

log_inference("\n" + "="*30 + " Sample Inference Logging Complete " + "="*30 + "\n")

Phase 1 Sample Inference Logging Started. Results will be saved to: /content/drive/MyDrive/Secure-Inference/phase1_results/phase1_non_secure_sample_inference.txt
Using device: cuda
DATA_ROOT for sample images: face_data/FaceClasses
Number of classes (NUM_CLASSES): 4
'val_tf' (validation transform) found in scope and will be used.
ResNet50 PyTorch model loaded from /content/drive/MyDrive/Secure-Inference/models/resnet50_mod.pth
SqueezeNet PyTorch model loaded from /content/drive/MyDrive/Secure-Inference/models/squeezenet_mod.pth
ONNX session created for ResNet50_FP32 using ['CUDAExecutionProvider', 'CPUExecutionProvider']
ONNX session created for ResNet50_FP16 using ['CUDAExecutionProvider', 'CPUExecutionProvider']
ONNX session created for SqueezeNet_FP32 using ['CUDAExecutionProvider', 'CPUExecutionProvider']
ONNX session created for SqueezeNet_FP16 using ['CUDAExecutionProvider', 'CPUExecutionProvider']

Using FIXED sample images: {'0': ['aa048t2aaunaff005.png', 'aa048t2aaunaff008.png